<a href="https://colab.research.google.com/github/m1kemp/School-Hero/blob/main/%CE%91%CE%BD%CF%84%CE%AF%CE%B3%CF%81%CE%B1%CF%86%CE%BF_DownLoadPhysionetDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import requests

# URL of PhysioNet dataset
DATA_URL_PHYSIONET = "https://physionet.org/static/published-projects/challenge-2017/1.0.0/training2017.zip"
SAVE_PATH_PHYSIONET = "data/training2017.zip"

# URL of MIT-BIH dataset
DATA_URL_MITBIH = "https://physionet.org/static/published-projects/mitdb/mit-bih-arrhythmia-database-1.0.0.zip"
SAVE_PATH_MITBIH = "data/mit-bih-arrhythmia-database-1.0.0.zip"

def download_data(DATA_URL, SAVE_PATH):
    if not os.path.exists("data"):
        os.makedirs("data")

    print("Downloading dataset...")
    response = requests.get(DATA_URL, stream=True)

    with open(SAVE_PATH, "wb") as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)

    print("Download complete! Unzipping...")
    os.system(f"unzip {SAVE_PATH} -d data/")
    print("Dataset is ready.")

    # Delete the zip file
    os.remove(SAVE_PATH)
    print(f"Deleted zip file: {SAVE_PATH}")

download_data(DATA_URL_PHYSIONET, SAVE_PATH_PHYSIONET)
download_data(DATA_URL_MITBIH, SAVE_PATH_MITBIH)


Download complete! Unzipping...
Dataset is ready.
Deleted zip file: data/training2017.zip
Download complete! Unzipping...
Dataset is ready.
Deleted zip file: data/mit-bih-arrhythmia-database-1.0.0.zip


In [2]:
!pip install wfdb
!pip install neurokit2
!pip install biosppy
!pip install torch
!pip install pickle
!pip install gzip
!pip install peakutils


import os
import numpy as np
import pandas as pd
import wfdb
import scipy.io
import scipy.signal
import neurokit2 as nk
import biosppy.signals.ecg as ecg
import torch
import pickle
import gzip

# Define constants
TARGET_SAMPLING_RATE = 125  # Hz
MAX_LEN_PHYSIONET = 10 * TARGET_SAMPLING_RATE  # 10 seconds
MAX_LEN_MITBIH = 30 * TARGET_SAMPLING_RATE  # 30 seconds

# Load PhysioNet dataset
def load_physionet_data(path):
    """Load PhysioNet 2017 dataset from .mat files and reference.csv."""
    signals, labels = [], []
    ref_df = pd.read_csv(os.path.join(path, "REFERENCE.csv"), header=None)
    ref_dict = dict(zip(ref_df[0], ref_df[1]))
    label_mapping = {"N": 0, "A": 1, "O": 2, "~": 3}  # Modify as per dataset classes


    for file in os.listdir(path):
        if file.endswith(".mat"):
            record_name = file.replace(".mat", "")
            mat_data = scipy.io.loadmat(os.path.join(path, file))
            signal = mat_data["val"][0]  # Extract ECG lead
            label = ref_dict.get(record_name, None)
            if label:
                signals.append(signal)
                physionet_labels = label_mapping[label]
                labels.append(physionet_labels)

    return signals, labels



# Downsampling function
def downsample_signal(signal, original_fs, target_fs=125):
    """Downsample ECG signal from original_fs to target_fs."""
    num_samples = int(len(signal) * target_fs / original_fs)
    return scipy.signal.resample(signal, num_samples)

# Normalize function
def normalize_signal(signal):
    """Normalize ECG signal between 0 and 1."""
    return (signal - np.min(signal)) / (np.max(signal) - np.min(signal))

# R-peak detection
def detect_r_peaks(signal, sampling_rate=125):
    """Detect R-peaks using the Pan-Tompkins algorithm."""
    _, r_peaks = nk.ecg_peaks(signal, sampling_rate=sampling_rate)
    return np.array(r_peaks["ECG_R_Peaks"])

# Beat segmentation
def extract_beats(signal, r_peaks, window_size=0.5, fs=125):
    """Extract ECG beats centered around R-peaks."""
    beat_length = int(window_size * fs)
    beats = []
    for peak in r_peaks:
        start = max(0, peak - beat_length)
        end = min(len(signal), peak + beat_length)
        beat = signal[start:end]
        beats.append(beat)
    return beats

# Zero-padding
def pad_signal(signal, max_len=MAX_LEN_PHYSIONET):
    """Pad signal to max_len with zeros."""
    if len(signal) < max_len:
        return np.pad(signal, (0, max_len - len(signal)), 'constant')
    else:
        return signal[:max_len]

# Full preprocessing pipeline
def preprocess_ecg_dataset(dataset_path):
    """Preprocess ECG dataset from PhysioNet or MIT-BIH."""
    signals, labels = load_physionet_data(dataset_path)
    original_fs = 300  # PhysioNet signals are sampled at 300 Hz
    max_len = MAX_LEN_PHYSIONET
    processed_signals,processed_labels = [],[]
    for signal,label in zip(signals,labels):
        # 1. Downsampling
        signal = downsample_signal(signal, original_fs, TARGET_SAMPLING_RATE)
        # 2. Normalization
        signal = normalize_signal(signal)
        # 3. R-peak detection
        r_peaks = detect_r_peaks(signal, TARGET_SAMPLING_RATE)
        # 4. Beat extraction
        beats = extract_beats(signal, r_peaks)
        # 5. Zero-padding each beat
        padded_beats = [pad_signal(beat, max_len) for beat in beats]
        processed_signals.extend(padded_beats)  # Collect all beats
        processed_labels.extend([label] * len(padded_beats))  # Assign the same label to all extracted beats

    return np.array(processed_signals), np.array(processed_labels)

# Example Usage
physionet_data, physionet_labels = preprocess_ecg_dataset("data/training2017")

# Convert to PyTorch Tensors
X_physionet = torch.tensor(physionet_data, dtype=torch.float32)
y_physionet = torch.tensor(physionet_labels, dtype=torch.long)

#save the processed data
# Save data with gzip compression
with gzip.open("pretraining_data.pkl.gz", "wb") as f:
    pickle.dump((X_physionet, y_physionet), f)



print(f"PhysioNet Data Shape: {X_physionet.shape}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.3/162.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 110.9 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 693.1/693.1 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.0/158.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━

In [3]:
%%writefile NNModel.py
import torch
import torch.nn as nn
import torch.nn.functional as F

# Squeeze-and-Excitation (SE) Module
class SEBlock(nn.Module):
    def __init__(self, channels, reduction=16):
        super(SEBlock, self).__init__()
        self.global_avg_pool = nn.AdaptiveAvgPool1d(1)  # Global Average Pooling
        self.fc1 = nn.Linear(channels, channels // reduction)
        self.fc2 = nn.Linear(channels // reduction, channels)

    def forward(self, x):
        batch, channels, _ = x.shape
        se = self.global_avg_pool(x).view(batch, channels)  # Global Avg Pool
        se = F.relu(self.fc1(se))  # First FC + ReLU
        se = F.softmax(self.fc2(se), dim=1)  # Second FC + Softmax
        se = se.view(batch, channels, 1)
        return x * se  # Scale the input

# Residual Block with SE Module
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=5):
        super(ResidualBlock, self).__init__()
        self.maxpool = nn.MaxPool1d(kernel_size=2)  # Add MaxPooling layer
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size, padding=kernel_size//2)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size, padding=kernel_size//2)
        self.se = SEBlock(out_channels)
        self.shortcut = nn.Conv1d(in_channels, out_channels, kernel_size=1) if in_channels != out_channels else nn.Identity()



    def forward(self, x):
        x= self.maxpool(x)  # Apply MaxPooling
        residual = self.shortcut(x)  # Skip connection
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.conv2(out)
        out = self.se(out)  # Apply SE Module
        out += residual  # Add Skip Connection
        return F.relu(out)

# Full Model: CNN + BiLSTM + FC
class ECGClassifier(nn.Module):
    def __init__(self, num_classes=5):
        super(ECGClassifier, self).__init__()

        # Initial Convolutional Layers
        self.conv1 = nn.Conv1d(1, 32, kernel_size=5, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.conv2 = nn.Conv1d(32, 32, kernel_size=5, padding=2)
        self.bn2 = nn.BatchNorm1d(32)

        # Residual Blocks with SE Module (4 Blocks)
        self.resblock1 = ResidualBlock(32, 64)
        self.resblock2 = ResidualBlock(64, 96)
        self.resblock3 = ResidualBlock(96, 128)
        self.resblock4 = ResidualBlock(128, 160)

        # MaxPooling Layer
        self.maxpool = nn.MaxPool1d(kernel_size=2)

        # BiLSTM Layers
        self.lstm = nn.LSTM(160, 64, num_layers=2, bidirectional=True, batch_first=True)
        self.dropout1 = nn.Dropout(0.2)
        self.dropout2 = nn.Dropout(0.5)

        # Fully Connected Layers
        self.fc1 = nn.Linear(128, 64)  # BiLSTM outputs 64*2=128
        self.fc2 = nn.Linear(64, num_classes)

    def forward(self, x):
         # First Conv -> ReLU -> BatchNorm
        x = self.conv1(x)
        x = F.relu(x)
        x = self.bn1(x)

        # Second Conv -> ReLU -> BatchNorm
        x = self.conv2(x)
        x = F.relu(x)
        x = self.bn2(x)

        # Residual Blocks with SE Module
        x = self.resblock1(x)
        x = self.resblock2(x)
        x = self.resblock3(x)
        x = self.resblock4(x)

        # Max Pooling
        x = self.maxpool(x)

        # Reshape for LSTM (Batch, SeqLen, Features)
        x = x.permute(0, 2, 1)

        # BiLSTM
        x, _ = self.lstm(x)
        x = self.dropout1(x[:, -1, :])  # Take last LSTM output
        x = F.relu(self.fc1(x))
        x = self.dropout2(x)
        x = self.fc2(x)

        return F.log_softmax(x, dim=1)  # Softmax for classification

# Create Model Instance
model = ECGClassifier(num_classes=5)

print(model)



Writing NNModel.py


In [1]:
!pip install nbimporter

import torch
import torch.nn as nn
import torch.optim as optim
import nbimporter
from NNModel import ECGClassifier
import pickle
import gzip

# Load Pretraining Data (PhysioNet)
with gzip.open("pretraining_data.pkl.gz", "rb") as f:
    X_train, y_train = pickle.load(f)

# Convert to PyTorch Dataset
class ECGDataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        self.X = X.unsqueeze(1)  # Add channel dimension
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Create DataLoader
batch_size = 64
train_dataset = ECGDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Modify Model for Pretraining (Last FC Layer = 4 Classes)
class PretrainECGClassifier(ECGClassifier):
    def __init__(self):
        super(PretrainECGClassifier, self).__init__(num_classes=4)  # Change output classes to 4

# Initialize Model
model = PretrainECGClassifier()

# Use CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model.to(device)

# He Normal Initialization
def he_init(m):
    if isinstance(m, nn.Linear) or isinstance(m, nn.Conv1d):
        nn.init.kaiming_normal_(m.weight, nonlinearity="relu")

model.apply(he_init)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Pretraining Loop
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

# Save Pretrained Model with gzip and pickle
with gzip.open("pretrained_model.pth.gz", "wb") as f:
    pickle.dump(model.state_dict(), f)
print("Pretraining Complete. Model Saved as pretrained_model.pth.gz!")

ECGClassifier(
  (conv1): Conv1d(1, 32, kernel_size=(5,), stride=(1,), padding=(2,))
  (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv1d(32, 32, kernel_size=(5,), stride=(1,), padding=(2,))
  (bn2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (resblock1): ResidualBlock(
    (maxpool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv1): Conv1d(32, 64, kernel_size=(5,), stride=(1,), padding=(2,))
    (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv1d(64, 64, kernel_size=(5,), stride=(1,), padding=(2,))
    (se): SEBlock(
      (global_avg_pool): AdaptiveAvgPool1d(output_size=1)
      (fc1): Linear(in_features=64, out_features=4, bias=True)
      (fc2): Linear(in_features=4, out_features=64, bias=True)
    )
    (shortcut): Conv1d(32, 64, kernel_size=(1,), stride=(1,))
  )
  (resblock2): ResidualBlock(
    

In [2]:
!zip -r training2017.zip data/training2017
!zip -r mit-bih-arrhythmia-database.zip data/mit-bih-arrhythmia-database-1.0.0

Η έξοδος ροής περικόπηκε στις τελευταίες 5000 γραμμές.
  adding: data/training2017/A04945.mat (deflated 30%)
  adding: data/training2017/A02342.mat (deflated 39%)
  adding: data/training2017/A00837.hea (deflated 13%)
  adding: data/training2017/A07788.hea (deflated 14%)
  adding: data/training2017/A04795.hea (deflated 12%)
  adding: data/training2017/A03440.mat (deflated 48%)
  adding: data/training2017/A04650.mat (deflated 42%)
  adding: data/training2017/A06246.hea (deflated 12%)
  adding: data/training2017/A02168.mat (deflated 23%)
  adding: data/training2017/A08450.mat (deflated 32%)
  adding: data/training2017/A02741.mat (deflated 46%)
  adding: data/training2017/A07582.hea (deflated 13%)
  adding: data/training2017/A04742.mat (deflated 24%)
  adding: data/training2017/A01205.hea (deflated 11%)
  adding: data/training2017/A00407.hea (deflated 12%)
  adding: data/training2017/A01161.hea (deflated 14%)
  adding: data/training2017/A02815.hea (deflated 14%)
  adding: data/training2017